### 📘 Notebook 03 — Feature Engineering & Clustering

In this notebook, we move from **data preparation** to **unsupervised learning**.  
Using the cleaned dataset (`books_final_1000.csv`), we’ll extract key features such as ratings, price, and genre to group similar books with **K-Means clustering**.

**Goals:**
- Prepare numerical and categorical features  
- Apply **K-Means** and evaluate with **Elbow Method** & **Silhouette Score**  
- Visualize and interpret clusters for future recommendations


### Step 1 — Imports & Setup

Import core libraries for clustering and visualization,  
reload the shared `functions.py` module, and verify that all paths from `config.yaml` are available.


In [ ]:
# ============================================================
# Step 1 — Imports & Setup 
# ============================================================

# --- System and project setup ---
import sys
from pathlib import Path

# Add 'notebooks' folder to path (functions.py lives there)
sys.path.append("notebooks")

# --- Load shared utilities ---
from functions import load_config, ensure_directories

# --- ML and visualization libraries ---
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# --- Visualization settings (como en clase) ---
sns.set(style="whitegrid", palette="muted")
plt.rcParams["figure.figsize"] = (8, 5)

# --- Load configuration and verify folders ---
config_path = Path("..") / "config.yaml"
config = load_config(config_path)
ensure_directories(config["paths"])

print("✅ Environment ready — config loaded and directories verified.")


### Step 2 — Load Final Dataset  

Load the cleaned and standardized dataset (`books_final_1000.csv`) generated in the previous notebook.  
We’ll inspect its structure, check column types, and verify that all key variables are ready for feature preparation.  


In [ ]:
# ============================================================
# Step 2 — Load Final Dataset
# ============================================================

import pandas as pd
from pathlib import Path

# --- Load dataset from data/clean ---
data_path = Path("..") / config["paths"]["data_clean"] / "books_final_1000.csv"
df = pd.read_csv(data_path)

print(f"✅ Dataset loaded successfully: {data_path}")
print(f"Shape: {df.shape}\n")

# --- Quick overview ---
display(df.head(5))

# --- Basic info and types ---
print("\n🔍 DataFrame Info:")
print(df.info())

# --- Missing values summary ---
missing_summary = df.isna().sum()
missing_summary = missing_summary[missing_summary > 0]

if not missing_summary.empty:
    print("\n⚠️ Missing values summary:")
    print(missing_summary)
else:
    print("\n✅ No missing values detected.")

# --- Optional: Unique values check (for categorical columns) ---
print("\n🧩 Unique values per column:")
print(df.nunique())


### Step 2.1 — Clean & Normalize Genres

In [ ]:
# ============================================================
# Step 2.1 — Clean & Normalize Genres
# ============================================================

"""
🎯 Ensure genre names are consistent and meaningful.
Preserve subcategories (e.g., Young Adult Fiction, Juvenile Fiction)
and fix missing or inconsistent values.
"""

import numpy as np

# --- Clean genre text ---
df["genre"] = (
    df["genre"]
    .astype(str)
    .str.strip()
    .replace({"nan": np.nan, "None": np.nan})
)

# --- Replace NaN with "Unknown" ---
df["genre"] = df["genre"].fillna("Unknown")

# --- Title case normalization ---
df["genre"] = df["genre"].str.title()

# --- Fix common variants ---
genre_replacements = {
    "Juvenile Fiction ": "Juvenile Fiction",
    "Young Adult Fiction ": "Young Adult Fiction",
    "Biography & Autobiography ": "Biography & Autobiography",
    "Nan": "Unknown"
}
df["genre"] = df["genre"].replace(genre_replacements)

# --- Final check ---
print("✅ Genre normalization complete.\n")
print("Top 10 genres after cleaning:")
display(df["genre"].value_counts().head(10))


### Step 2.2 — Consolidate Main Genres (for Descriptive Analysis)

Before running clustering, we standardize and consolidate similar genre labels
(e.g., Juvenile Fiction and Young Adult Fiction) under their main categories.

This step does not affect the clustering model directly,
but ensures cleaner genre information for later descriptive and visualization steps
(e.g., identifying dominant genres within each cluster).

In [ ]:
# ============================================================
# Step 2.2 — Clean and Normalize Genres Before Encoding
# ============================================================

def normalize_genre(value):
    if pd.isna(value):
        return "Unknown"
    value = value.strip().title()
    # Simplify subcategories
    if "Juvenile" in value:
        return "Juvenile Fiction"
    if "Young Adult" in value:
        return "Young Adult Fiction"
    if "Biography" in value:
        return "Biography & Autobiography"
    if "Comics" in value or "Graphic" in value:
        return "Comics & Graphic Novels"
    if "Poet" in value:
        return "Poetry"
    if "Drama" in value:
        return "Drama"
    if "Relig" in value:
        return "Religion"
    if "History" in value:
        return "History"
    if "Fiction" in value:
        return "Fiction"
    return value

# Apply normalization
df["genre"] = df["genre"].apply(normalize_genre)

print("✅ Genres normalized successfully.\n")
print("Top 10 genres after normalization:")
display(df["genre"].value_counts().head(10))


### Step 3 — Feature Preparation

We now prepare the numerical features for K-Means clustering.

- Selected features: **avg_rating** and **price**
- Missing prices are filled with the median value
- Features are standardized using **StandardScaler**
- The resulting matrix `X` will be used for clustering


In [ ]:
# ============================================================
# Step 3 — Feature Preparation (Numeric Features Only)
# ============================================================

from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

# --- Select relevant numerical columns for clustering ---
features = ["avg_rating", "price"]
df_features = df[features].copy()

# --- Handle missing prices ---
median_price = df_features["price"].median()
df_features["price"] = df_features["price"].fillna(median_price)

# --- Reflect the filled prices back into the main DataFrame ---
df["price"] = df["price"].fillna(median_price)
print(f"Filled missing 'price' values with median: {median_price:.2f}")

# --- Standardize numeric columns ---
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df_features)

# --- Convert back to DataFrame for easier handling ---
df_encoded = pd.DataFrame(df_scaled, columns=features)

# --- Sanity check ---
missing_check = df_encoded.isna().sum().sum()
if missing_check == 0:
    print("✅ No missing values remain in feature matrix.")
else:
    print(f"⚠️ {missing_check} missing values still present — check source data.")

# --- Assign to feature matrix for clustering ---
X = df_encoded.values

print(f"\n✅ Feature matrix ready for clustering. Shape: {df_encoded.shape}")

# --- Quick preview ---
display(df_encoded.head(5))


## Step 4 — K-Means Clustering (Elbow & Silhouette Method)

In this step, we apply the K-Means algorithm to identify potential groups (clusters) among the books.

We will:
1. Run K-Means for different values of *k* (from 2 to 10)
2. Record both the **Inertia** (Elbow Method) and the **Silhouette Score**
3. Identify the optimal number of clusters (`best_k`) based on the highest silhouette score
4. Visualize both metrics side by side for comparison

The **Elbow Method** helps detect the point where adding more clusters no longer improves the model significantly,  
while the **Silhouette Score** evaluates how well-separated the clusters are (higher values indicate better-defined clusters).

Finally, both the visualizations and metrics are saved for later analysis.


In [ ]:
# ============================================================
# Step 4 — K-Means Clustering (Elbow & Silhouette Method) — Fixed Silhouette Range
# ============================================================

import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
from pathlib import Path
import pandas as pd

# --- Feature matrix (numeric only) ---
X = df_encoded.values

# --- Initialize lists ---
inertias = []
silhouette_scores = []
K_range = range(2, 11)

print("Running K-Means for k = 2 to 10...\n")

# --- Run K-Means across different k values ---
for k in K_range:
    try:
        kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
        kmeans.fit(X)
        inertias.append(kmeans.inertia_)
        score = silhouette_score(X, kmeans.labels_)
        silhouette_scores.append(score)
        print(f"k={k} — Inertia={kmeans.inertia_:.2f}, Silhouette={score:.4f}")
    except Exception as e:
        print(f"⚠️ Error for k={k}: {e}")
        inertias.append(np.nan)
        silhouette_scores.append(np.nan)

# --- Determine best k by silhouette score ---
valid_scores = [s for s in silhouette_scores if not np.isnan(s)]
best_k = list(K_range)[silhouette_scores.index(max(valid_scores))]

# --- Show top 3 silhouette values ---
sorted_scores = sorted(zip(K_range, silhouette_scores), key=lambda x: x[1], reverse=True)
print("\nTop 3 silhouette scores:")
for i, (k_val, s_val) in enumerate(sorted_scores[:3], start=1):
    print(f"{i}. k={k_val} → silhouette={s_val:.4f}")

print(f"\n✅ Best k by silhouette score: {best_k}")

# ============================================================
# Visualization — Elbow & Silhouette (Range -1 to 1)
# ============================================================

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(11, 4))

# --- Elbow Method ---
ax1.plot(K_range, inertias, marker='o', color='steelblue')
ax1.set_title("Elbow Method — K-Means Inertia", fontsize=11)
ax1.set_xlabel("Number of Clusters (k)")
ax1.set_ylabel("Inertia")
ax1.grid(alpha=0.3, linestyle='--')

# --- Silhouette Scores ---
ax2.plot(K_range, silhouette_scores, marker='o', color='orange')
ax2.set_title("Silhouette Scores (Range -1 to 1)", fontsize=11)
ax2.set_xlabel("Number of Clusters (k)")
ax2.set_ylabel("Silhouette Score")
ax2.set_ylim(-1, 1)  # <-- ✅ Force the correct range
ax2.axhline(0, color='gray', linestyle='--', linewidth=0.8)
ax2.grid(alpha=0.3, linestyle='--')

plt.tight_layout()
plt.show()

# ============================================================
# Save Results
# ============================================================

viz_path = Path("..") / "visualizations"
viz_path.mkdir(parents=True, exist_ok=True)
fig.savefig(viz_path / "kmeans_elbow_silhouette_fixed.png", dpi=300, bbox_inches="tight")

metrics_path = Path("..") / "data" / "clean"
metrics_path.mkdir(parents=True, exist_ok=True)

metrics_df = pd.DataFrame({
    "k": list(K_range),
    "inertia": inertias,
    "silhouette": silhouette_scores
})
metrics_df.to_csv(metrics_path / "kmeans_metrics_fixed.csv", index=False, encoding="utf-8-sig")

print(f"\n💾 Results saved → {metrics_path / 'kmeans_metrics_fixed.csv'}")
print(f"🏷️ Best number of clusters: {best_k}")



###  Interpretation of Clustering Metrics


The Elbow Method shows a strong drop in inertia from k = 2 to k = 3, after which the curve flattens — suggesting diminishing returns in compactness beyond three clusters.

The Silhouette Scores (ranging from –1 to 1) indicate that k = 2 achieves the highest separation (≈ 0.74), meaning clusters are more compact and distinct.
However, this two-cluster model oversimplifies the data, grouping too many diverse books together.

📊 Decision: Although the silhouette metric suggests k = 2, we selected k = 3 as the final configuration — providing a more interpretable segmentation that better reflects real differences in rating and price (e.g., affordable, mid-range, and premium books).


## Step 5 — Apply Final K-Means & Visualize Clusters (PCA 2D)

After determining the optimal number of clusters (`k=2`), we apply the final K-Means model to assign each book to a specific cluster.

Steps performed:
1. Scale all numeric features for consistent distance calculation.
2. Train the K-Means model using the selected value of *k*.
3. Assign the resulting cluster labels to each book.
4. Apply **Principal Component Analysis (PCA)** to reduce the feature space to two dimensions for visualization.
5. Plot the resulting clusters using a 2D scatter plot, where each point represents a book and color indicates its cluster.

The resulting visualization helps identify distinct groups of books based on their ratings, price levels, and genre characteristics.
Both the model output and visualization are saved for further analysis.


In [ ]:
# ============================================================
# Step 5 — Apply Final K-Means & Visualize Clusters (PCA 2D, k = 3)
# ============================================================

from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import numpy as np

# --- Final number of clusters ---
k_final = 3
print(f"Applying final K-Means model with k = {k_final}...\n")

# --- Use scaled numeric features from df_encoded (already standardized) ---
X_scaled = df_encoded.values

# --- Train K-Means ---
kmeans_final = KMeans(n_clusters=k_final, random_state=42, n_init=10)
cluster_labels = kmeans_final.fit_predict(X_scaled)

# --- Assign clusters safely ---
df = df.copy()
df["cluster"] = cluster_labels

# --- PCA for visualization ---
pca = PCA(n_components=2, random_state=42)
pca_components = pca.fit_transform(X_scaled)
df["pca_1"] = pca_components[:, 0]
df["pca_2"] = pca_components[:, 1]

# ============================================================
# Visualization — PCA 2D Scatter Plot (k = 3)
# ============================================================

fig_pca, ax = plt.subplots(figsize=(8, 6))
sns.scatterplot(
    data=df,
    x="pca_1", y="pca_2",
    hue="cluster",
    palette="Set2",
    s=65,
    alpha=0.85,
    edgecolor="white",
    linewidth=0.7,
    ax=ax
)
ax.set_title(f"Book Clusters — PCA 2D Projection (k = {k_final})", fontsize=13, pad=10)
ax.set_xlabel("Principal Component 1")
ax.set_ylabel("Principal Component 2")
ax.legend(title="Cluster", loc="best", fontsize=9)
ax.grid(alpha=0.25, linestyle="--")
plt.tight_layout()
plt.show()

# ============================================================
# Save Plot
# ============================================================

viz_path = Path("..") / "visualizations"
viz_path.mkdir(parents=True, exist_ok=True)
fig_pca.savefig(viz_path / f"pca_clusters_k{k_final}.png", dpi=300, bbox_inches="tight")

print(f"Clustering completed. {df['cluster'].nunique()} clusters created.")
print(f"PCA cluster visualization saved → {viz_path / f'pca_clusters_k{k_final}.png'}\n")

# --- Preview sample ---
display(df[["title", "author", "avg_rating", "genre", "price", "cluster"]].head(10))


### 📊 Interpretation of Final Clusters (k = 3)

The **3-cluster configuration** reveals three meaningful groups of books based primarily on their **average rating** and **price level**.  
Although genre information wasn’t directly used in the clustering, it helps describe each group’s general tendencies.

---

 **Cluster 0 (Green — Affordable / Mainstream)**
- Contains books with **lower prices** (mostly below €10).  
- ⭐ Average ratings remain strong (≈ 4.0–4.2), suggesting consistent reader satisfaction despite lower cost.  
- Genres are mainly **fiction and young adult**, representing **popular mainstream titles**.

---

**Cluster 1 (Blue — Mid-Range / Balanced)**
- Represents books with **moderate prices** and **balanced ratings** (≈ 4.1).  
- Combines both **fiction** and **non-fiction** categories, offering a diverse yet stable segment.  
- This group likely includes **widely read and accessible titles** with broad appeal.

---

**Cluster 2 (Orange — Premium / Niche)**
- Features **higher-priced titles**, often above €25.  
- ⭐ Ratings remain solid (≈ 4.0), but the group shows greater price variability.  
- Genres tend to include **art, criticism, or specialized works**, indicating **premium or niche markets**.

---

**Interpretation**

The model now differentiates books by **price tiers** and **market positioning**:  

- **Cluster 0 →** Affordable, mainstream fiction — broad audience appeal.  
- **Cluster 1 →** Balanced, mid-range titles — mix of genres with stable quality.  
- **Cluster 2 →** Premium, niche books — higher prices, specialized interest.  

This 3-cluster segmentation provides richer insight for future **recommendation logic**,  
allowing readers to explore books within their preferred **value, genre, or quality range**.


### Step 5.1 — Analyze Cluster Centroids

Let’s inspect the numerical centroids of each cluster to understand
how books are grouped — for example, by their average rating or price.


In [ ]:
# ============================================================
# Step 5.2 — Cluster Composition Summary (k = 3)
# ============================================================

"""
🎯 Step 5.2 — Analyze Cluster Composition by Genre (k = 3)
This version summarizes how genres distribute within each cluster,
while also showing the average rating and price per cluster.
"""

# --- Numeric centroids (average rating & price) ---
cluster_centroids = (
    df.groupby("cluster", as_index=False)
    .agg({
        "avg_rating": "mean",
        "price": "mean"
    })
    .round(2)
)

# --- Genre distribution within each cluster ---
genre_distribution = (
    df.groupby(["cluster", "genre"])
    .size()
    .reset_index(name="count")
)

# --- Calculate proportions per cluster ---
cluster_sizes = df["cluster"].value_counts().to_dict()
genre_distribution["proportion_%"] = genre_distribution.apply(
    lambda row: round((row["count"] / cluster_sizes[row["cluster"]]) * 100, 2),
    axis=1
)

# --- Merge numeric averages with genre distribution ---
cluster_summary = (
    genre_distribution.merge(cluster_centroids, on="cluster", how="left")
    .sort_values(["cluster", "count"], ascending=[True, False])
)

# --- Display top 5 genres per cluster ---
print("📊 Cluster Composition Summary (Top 5 Genres per Cluster, k = 3):\n")
display(cluster_summary.groupby("cluster").head(5))

print("\n🧭 Interpretation Guide:")
print("- avg_rating → Average rating within the cluster.")
print("- price → Mean price (EUR) within the cluster.")
print("- count → Number of books belonging to that genre in the cluster.")
print("- proportion_% → Relative share (%) of that genre within the cluster.")


### 📚 Interpretation — Cluster Composition by Genre (k = 3)

The genre distribution confirms a clear segmentation of the book market into three tiers based on price and reader ratings:

Cluster 0 represents the mainstream and affordable segment — books priced around €8.28, with strong average ratings (≈ 3.87 ★), mostly belonging to fiction and young adult genres.

Cluster 1 captures the mid-range market, combining both fiction and non-fiction titles. These books maintain balanced ratings (≈ 4.19) and moderate prices around €8.89, reflecting broadly popular and accessible works.

Cluster 2 consists of premium and niche titles, typically priced around €43, including art, criticism, and specialized literature. Despite their higher price, these books sustain solid reader satisfaction, highlighting their collector or expert appeal (around 4.21★).

Overall, even though genre was not included as a clustering feature, its distribution reveals how the clusters naturally align with economic and thematic segmentation — from accessible mainstream fiction to high-value specialized works.

## Step 6 — Export Final Clustered Dataset

We’ll now export the final dataset including the cluster labels (`cluster`)
and PCA coordinates (`pca_1`, `pca_2`) for visualization and further analysis.  
This dataset can be used in **Tableau**, **Power BI**, or in the next notebook
for building the **Recommendation System**.


In [ ]:
# ============================================================
# Step 6 — Export Final Clustered Dataset & Cluster Summary (Overwrite Only)
# ============================================================

from pathlib import Path
import pandas as pd

# --- Define export paths (overwrite mode) ---
data_clean_path = Path("..") / "data" / "clean"
viz_path = Path("..") / "visualizations"
data_clean_path.mkdir(parents=True, exist_ok=True)
viz_path.mkdir(parents=True, exist_ok=True)

# --- Export final dataset (overwrite previous file) ---
final_cluster_path = data_clean_path / "books_clustered_final.csv"

export_cols = [
    "title", "author", "avg_rating", "genre", "price", "currency",
    "cover_url", "link", "cluster", "pca_1", "pca_2"
]

df[export_cols].to_csv(final_cluster_path, index=False, encoding="utf-8-sig")
print(f"💾 Final clustered dataset (k = 3) overwritten successfully → {final_cluster_path.resolve()}")

print("\n📘 Sample of exported dataset:")
display(df[export_cols].head(10))

# ============================================================
# 📊 Cluster Summary (genre proportions + numeric averages)
# ============================================================

# --- Numeric stats per cluster ---
cluster_centroids = (
    df.groupby("cluster", as_index=False)
    .agg({
        "avg_rating": "mean",
        "price": "mean"
    })
    .round(2)
)

# --- Genre distribution per cluster ---
genre_distribution = (
    df.groupby(["cluster", "genre"])
    .size()
    .reset_index(name="count")
)

# --- Add proportions per cluster ---
cluster_sizes = df["cluster"].value_counts().to_dict()
genre_distribution["proportion_%"] = genre_distribution.apply(
    lambda row: round((row["count"] / cluster_sizes[row["cluster"]]) * 100, 2),
    axis=1
)

# --- Merge numeric stats with genre composition ---
cluster_summary = genre_distribution.merge(cluster_centroids, on="cluster", how="left")
cluster_summary = cluster_summary.sort_values(["cluster", "count"], ascending=[True, False])

print("\n📗 Cluster Summary (Top Genres per Cluster, k = 3):")
display(cluster_summary.groupby("cluster").head(5))

# ✅ No new CSV created — previous files remain updated
